## 자동차 번호판 검출하기

In [45]:
# 라이브러리 불러오기
import cv2
import matplotlib.pyplot as plt

import numpy as np
import time

In [46]:
import datetime
import PIL 
from PIL import Image

In [48]:
# 전체 이미지 불러올 경로 지정
img_paths = glob.glob('data/img/*.jpg') # glob 함수로 폴더 내 모든 jpg 파일을 불러옴
for img_path in img_paths: # img_paths에서 파일 경로 하나씩 꺼내오기
    img = cv2.imread(img_path) # 해당 경로에 있는 jpg 이미지 불러오기
    
    height, width, channel = img.shape # 뒤에서 쓸 높이, 너비, 채널 변수 설정
    
    # 그레이스케일 적용
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Thresholding 하기
    blurred = cv2.GaussianBlur(gray, (3,3), 0) # 가우시안 블러링 
    img_th = cv2.adaptiveThreshold(blurred,
                              maxValue=255.0,
                              adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                              thresholdType=cv2.THRESH_BINARY_INV,
                              blockSize=19,
                              C=9) 

    # 해당 이미지의 모든 Contour 검출
    _, cnts, _=cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    result = np.zeros((height, width, channel), dtype=np.uint8)
    
    # Contour 그려주기
    cv2.drawContours(result, contours=cnts, contourIdx=-1, color=(255, 0, 255))

    # Contour 사각형 그려주기
    result = np.zeros((height, width, channel), dtype=np.uint8)

    cnts_dict = []

    for cnt in cnts: # Contour 하나씩 꺼내기
        x, y, w, h = cv2.boundingRect(cnt) # 컨투어에 외접한 직사각형 좌표(x, y), 높이, 너비 리턴
        cv2.rectangle(result, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 255), thickness=2)
        # 해당 좌표를 따라 사각형 그리기

        cnts_dict.append({
            'contour':cnt,
            'x':x,
            'y':y,
            'w':w,
            'h':h,
            'cx':x + (w/2),
            'cy':y + (h/2)}) # 그려진 사각형들의 컨투어와 좌표를 cnts_dict에 추가

    # 불필요한 contour 솎아내기
    MIN_AREA = 80
    MIN_WIDTH, MIN_HEIGHT = 2, 8
    MIN_RATIO, MAX_RATI0 = 0.25, 1.0

    possible_cnts = []

    cnt = 0
    for d in cnts_dict:
        area = d['w']*d['h']
        ratio = d['w']/d['h']

        if area > MIN_AREA \
        and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
        and MIN_RATIO < ratio < MAX_RATI0:
            d['idx'] = cnt
            cnt += 1
            possible_cnts.append(d)

    result = np.zeros((height, width, channel), dtype=np.uint8)

    for d in possible_cnts:
        cv2.rectangle(result, 
                      pt1=(d['x'], d['y']), 
                      pt2=(d['x']+d['w'], d['y']+d['h']),
                      color=(255, 0 , 255),
                      thickness=2)
    
    # 차량 번호판 contour 검출하기
    MAX_DIAG_MULTIPLYER = 5
    MAX_ANGLE_DIFF = 12.0
    MAX_AREA_DIFF = 0.5
    MAX_WIDTH_DIFF = 0.8
    MAX_HEIGHT_DIFF = 0.2
    MIN_N_MATCHED = 3

    def find_chars(cnt_list):
        matched_result_idx = []

        for d1 in cnt_list:
            matched_cnts_idx = []
            for d2 in cnt_list:
                if d1['idx'] == d2['idx']:
                    continue

                dx = abs(d1['cx'] - d2['cx'])
                dy = abs(d1['cy'] - d2['cy'])

                diagonal_length1 = np.sqrt(d1['w']**2 + d1['h']**2)
                distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
                if dx == 0:
                    angle_diff = 90
                else:
                    angle_diff = np.degrees(np.arctan(dy/dx))

                area_diff = abs(d1['w']*d1['h'] - d2['w']*d2['h']) / (d1['w']*d1['h'])
                width_diff = abs(d1['w'] - d2['w']) / d1['w']
                height_diff = abs(d1['h'] - d2['h']) / d1['h']

                if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
                and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
                and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                    matched_cnts_idx.append(d2['idx'])

            matched_cnts_idx.append(d1['idx'])

            if len(matched_cnts_idx) < MIN_N_MATCHED:
                continue

            matched_result_idx.append(matched_cnts_idx)

            unmatched_cnt_idx = []
            for d4 in cnt_list:
                if d4['idx'] not in matched_cnts_idx:
                    unmatched_cnt_idx.append(d4['idx'])

            unmatched_cnt = np.take(possible_cnts, unmatched_cnt_idx)

            recursive_cnt_list = find_chars(unmatched_cnt)

            for idx in recursive_cnt_list:
                matched_result_idx.append(idx)

            break

        return matched_result_idx

    result_idx = find_chars(possible_cnts)

    matched_result = []
    for idx_list in result_idx:
        matched_result.append(np.take(possible_cnts, idx_list))

    result = np.zeros((height, width, channel), dtype=np.uint8)

    for r in matched_result:
        for d in r:
            cv2.rectangle(result, 
                          pt1=(d['x'], d['y']), 
                          pt2=(d['x']+d['w'], d['y']+d['h']), 
                          color=(255, 0, 255),
                         thickness=2)

    # 차량 번호판 부분만 crop 하기
    PLATE_WIDTH_PADDING = 1.3
    PLATE_HEIGHT_PADDING = 1.5
    MIN_PLATE_RATIO = 3
    MAX_PLATE_RATIO = 10

    plate_imgs = []
    plate_infos = []

    for i, matched_chars in enumerate(matched_result):
        sorted_chars = sorted(matched_chars, key=lambda x:x['cx'])

        plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
        plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2

        plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x'])*PLATE_WIDTH_PADDING

        sum_height = 0
        for d in sorted_chars:
            sum_height += d['h']

        plate_height = int(sum_height / len(sorted_chars)*PLATE_HEIGHT_PADDING)

        triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
        triangle_hypotenus = np.linalg.norm(np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) - 
                                           np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']]))

        angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))

        rotation_mtrx = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)
        img_rotated = cv2.warpAffine(img_th, M=rotation_mtrx, dsize=(width, height))

        img_cropped = cv2.getRectSubPix(img_rotated,
                                       patchSize = (int(plate_width), int(plate_height)), 
                                       center=(int(plate_cx), int(plate_cy)))

        if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO \
        or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
            continue

        plate_imgs.append(img_cropped)
        plate_infos.append({
            'x':int(plate_cx - plate_width / 2),
            'y':int(plate_cy - plate_height / 2),
            'w':int(plate_width),
            'h':int(plate_height)})
        
        now = datetime.datetime.now()
        now_str = now.strftime("%H.%M.%S.%f")
        finished = Image.fromarray(img_cropped)
        finished.save("data/num/{}.jpg".format(now_str))

In [61]:
# # 추출된 글자 전처리(Extra)
# img_result = cv2.GaussianBlur(img_cropped, (5,5), 0)
# _, img_result = cv2.threshold(img_result, 
#                               thresh=0.0, 
#                               maxval=255.0, 
#                               type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# img_result = cv2.copyMakeBorder(img_result, 
#                                 top=0, 
#                                 bottom=0, 
#                                 left=0, 
#                                 right=0,
#                                borderType=cv2.BORDER_CONSTANT, 
#                                 value=(0,0,0))

# plt.imshow(img_result, cmap='gray')
# plt.axis('off')
# plt.show()

In [12]:
# # pytesseract로 글자 추출 결과 확인
# import PIL
# import pytesseract

# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

# text = pytesseract.image_to_string(img_result, lang='eng')

# result_texts = ''
# has_digit = False

# for n in text:
#     if ord('A') <= ord(n) <= ord('Z') or n.isdigit():
#         if n.isdigit():
#             has_digit = True
#             result_texts += n
    
# print(result_texts)

# 자투리 코드
#         fnames = ['plate.{}.jpg'.format(i) for i in range(503)]
#         for fname in fnames:
#             src = os.path.join(original_dataset_dir, fname)
#             dst = os.path.join(num_dir, fname)
#             shutil.copyfile(src, dst)
        
#         cv2.imwrite('cropped18.jpg', img_cropped)
#         plt.subplot(len(matched_result), 1, i+1)
#         plt.axis('off')
#         plt.imshow(img_cropped, cmap='gray')

## 이미지 검색기 만들기

In [109]:
# opencv 버전 확인
cv2.__version__

'3.4.2'

In [113]:
!pip install utils

In [114]:
#라이브러리 불러오기

import glob
import numpy as np
import cv2
import random
import os

ModuleNotFoundError: No module named 'utils.featureextractor'